### Parses PDF files and extracts numbered items from them into a list and then creates a github project with the list of items as issues.

This notebook reuses the code from the following notebooks:
- [ColPali_+_Qwen2_VL.ipynb](https://github.com/merveenoyan/smol-vision/blob/main/ColPali_%2B_Qwen2_VL.ipynb)

In [ ]:
%pip install requests
%pip install python-dotenv
%pip install --upgrade byaldi

# %sudo apt-get install -y poppler-utils # is there a mac equivalent?

%pip install -q pdf2image git+https://github.com/huggingface/transformers.git qwen-vl-utils flash-attn


In [ ]:

import os
import json
import torch
import requests
from dotenv import load_dotenv
from byaldi import RAGMultiModalModel
from pdf2image import convert_from_path
from huggingface_hub import notebook_login
from qwen_vl_utils import process_vision_info
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor

notebook_login()

# Load environment variables from .env file
load_dotenv()

Owner ID: MDQ6VXNlcjI2MzU4MDQ3


In [ ]:
import os

asset_dir = "assets/"

files = [f for f in os.listdir(asset_dir) if os.path.isfile(os.path.join(asset_dir, f))]

print(files)

images = convert_from_path(asset_dir + files[0]) # TODO: loop through all files
images[5]

In [ ]:

RAG = RAGMultiModalModel.from_pretrained("vidore/colpali")

In [ ]:
RAG.index(
    input_path= f"{asset_dir}{files[0]}", # path to the input file
    index_name="image_index", # index will be saved at index_root/index_name/
    store_collection_with_index=False,
    overwrite=True
)

In [ ]:
text_query = "How many issues are there in the image?"
results = RAG.search(text_query, k=1)
results

In [ ]:


model = Qwen2VLForConditionalGeneration.from_pretrained("Qwen/Qwen2-VL-2B-Instruct",
                                                        trust_remote_code=True, torch_dtype=torch.bfloat16).cuda().eval()

In [ ]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", trust_remote_code=True)

image_index = results[0]["page_num"] - 1
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": images[image_index],
            },
            {"type": "text", "text": text_query},
        ],
    }
]

In [ ]:

text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)


In [ ]:
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [ ]:
generated_ids = model.generate(**inputs, max_new_tokens=50)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

In [ ]:
print(output_text)

In [ ]:
# Replace these variables with your own values
github_token =  os.getenv('GITHUB_PROJECT_SCOPE_PAT')
github_owner = os.getenv('GITHUB_OWNER')


token = github_token
owner = github_owner
project_name = 'First Project'
issue_title = 'Issue Title'
issue_body = 'Issue Body'

# Get the node ID of the owner
owner_url = f'https://api.github.com/users/{owner}'
headers = {
    'Authorization': f'token {token}',
    'Accept': 'application/vnd.github+json'
}
response = requests.get(owner_url, headers=headers)
owner_id = response.json()['node_id']
print("Owner ID:", owner_id)

In [ ]:

# Create the project
create_project_query = {
    'query': f'mutation {{createProjectV2(input: {{ownerId: "{owner_id}", title: "{project_name}"}}) {{projectV2 {{id}}}}}}'
}
response = requests.post('https://api.github.com/graphql', headers=headers, data=json.dumps(create_project_query))
print(response.json())
project_id = response.json()['data']['createProjectV2']['projectV2']['id']
print("Project ID:", project_id)

In [14]:
assignees = ['tunji17']  # GitHub usernames of the assignees

# Get the node IDs of the assignees
assignee_ids = []
for assignee in assignees:
    assignee_url = f'https://api.github.com/users/{assignee}'
    response = requests.get(assignee_url, headers=headers)
    assignee_ids.append(response.json()['node_id'])

assignees_str = ', '.join([f'"{assignee_id}"' for assignee_id in assignee_ids])

In [ ]:

# Add a draft issue to the project
add_issue_query = {
    'query': f'mutation {{addProjectV2DraftIssue(input: {{projectId: "{project_id}", title: "{issue_title}", body: "{issue_body}", assigneeIds: [{assignees_str}]}}) {{projectItem {{id}}}}}}'
}
response = requests.post('https://api.github.com/graphql', headers=headers, data=json.dumps(add_issue_query))
print(response.json())